# Solve fluid diffusion equation in a rectangular domain with imposed flux on L_left

In [1]:
import numpy as np
from mpi4py import MPI
import pyvista
import ufl

from dolfinx import fem, plot
from dolfinx.io import XDMFFile
from dolfinx.fem import FunctionSpace, Constant, Function

from petsc4py import PETSc
from petsc4py.PETSc import ScalarType

## Read the mesh

In [2]:
# Read the mesh
with XDMFFile(MPI.COMM_WORLD, "mesh_rectangle/mesh.xdmf", "r") as xdmf:
    mesh = xdmf.read_mesh(name="Grid")
    cell_tags = xdmf.read_meshtags(mesh, name="Grid")
mesh.topology.create_connectivity(mesh.topology.dim-1, mesh.topology.dim)

with XDMFFile(MPI.COMM_WORLD, "mesh_rectangle/facet_mesh.xdmf", "r") as xdmf:
    facet_tags = xdmf.read_meshtags(mesh, name="Grid")

## Plot the mesh

In [3]:
topology, cell_types, geometry = plot.create_vtk_mesh(mesh, mesh.topology.dim)
grid = pyvista.UnstructuredGrid(topology, cell_types, geometry)

pyvista.set_jupyter_backend("pythreejs")

plotter = pyvista.Plotter()
plotter.add_mesh(grid, show_edges=True)
plotter.view_xy()
plotter.show()

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(color='#fefefe', inten…

## Define temporal parameters

In [4]:
t = 0 # Start time
T = 100000.0 # Final time
num_steps = 61     
dt = T / num_steps # time step size

## Finite element function space

In [5]:
V = FunctionSpace(mesh, ("CG", 1))  # Lagrange element and and linear elements (degree 1)

## Set initial conditions

In [6]:
p_n = Function(V)
p_n.name = "p_n"

## Boundary conditions

In [7]:
# DIRICHLET: p=0 on all Dirichlet BC (sides 1, 2 and 4)
boundary_dofs = fem.locate_dofs_topological(V, mesh.topology.dim-1, facet_tags.indices[facet_tags.values != 3])
bc = fem.dirichletbc(ScalarType(0), boundary_dofs, V)

# NEUMANN: dp/dt=-10 on the left side (side 3)
# we can define it as a function g=x-10 defined everywhere in the domain but that will be evaluated only on Neumann boundaries. Thus, g=-10 on side 3 and g=0 on side 4 (because x=10)
x = ufl.SpatialCoordinate(mesh)
g = x[0]-10

## Time-dependent output

In [8]:
p_h = Function(V)
p_h.name = "p_h"

## Trial and test functions

In [9]:
p, r = ufl.TrialFunction(V), ufl.TestFunction(V)

## Source term

In [10]:
# here f=0 as there is no source term
f = fem.Constant(mesh, ScalarType(0))

## Variational problem

As the test function is zero on the boundary integrals over the Dirichlet boundary, we can integrate g*v*ds over the entire boundary.

In [11]:
S = fem.Constant(mesh, ScalarType(1))
kappa = fem.Constant(mesh, ScalarType(1))

In [12]:
a = S * p * r * ufl.dx + dt * kappa * ufl.dot(ufl.grad(p), ufl.grad(r)) * ufl.dx 
L = (S * p_n + dt * f) * r * ufl.dx - dt * g * r * ufl.ds

#a = u * v * ufl.dx + dt*ufl.dot(ufl.grad(u), ufl.grad(v)) * ufl.dx 
#L = (u_n + dt * f) * v * ufl.dx

## Preparation of linear algebra structures for time dependent problems

In [13]:
bilinear_form = fem.form(a)
linear_form = fem.form(L)

In [14]:
# bilinear_form (a) isn't time dependant so we can assemble it once (we could do the same for the integral of the Neumann term as the flux is contant, but for now let's stay like that)
A = fem.petsc.assemble_matrix(bilinear_form, bcs=[bc])
A.assemble()
b = fem.petsc.create_vector(linear_form)

## Linear solver

In [15]:
# here matrix A is already assembled so we don't need to use petsc.LinearProblem like in tuto1
solver = PETSc.KSP().create(mesh.comm)
solver.setOperators(A)
solver.setType(PETSc.KSP.Type.PREONLY)
solver.getPC().setType(PETSc.PC.Type.LU)

In [16]:
import pyvista
pyvista.set_jupyter_backend("ipygany")

grid = pyvista.UnstructuredGrid(*plot.create_vtk_mesh(V))

def plot_function(t, uh):
    """
    Create a figure of the concentration uh warped visualized in 3D at timet step t.
    """
    p = pyvista.Plotter()
    # Update point values on pyvista grid
    grid.point_data[f"u({t})"] = uh.x.array.real
    # Warp mesh by point values
    warped = grid.warp_by_scalar(f"u({t})", factor=1.5)

    # Add mesh to plotter and visualize in notebook or save as figure
    actor = p.add_mesh(warped)
    if not pyvista.OFF_SCREEN:
        p.show()
    else:
        pyvista.start_xvfb()
        figure_as_array = p.screenshot(f"diffusion_{t:.2f}.png")
        # Clear plotter for next plot
        p.remove_actor(actor)
plot_function(0, p_h)

AppLayout(children=(VBox(children=(HTML(value='<h3>u(0)</h3>'), Dropdown(description='Colormap:', options={'Br…

In [17]:
for i in range(num_steps):
    t += dt

    # Update the right hand side reusing the initial vector
    with b.localForm() as loc_b:
        loc_b.set(0)
    fem.petsc.assemble_vector(b, linear_form)
    
    # Apply Dirichlet boundary condition to the vector
    fem.petsc.apply_lifting(b, [bilinear_form], [[bc]])
    b.ghostUpdate(addv=PETSc.InsertMode.ADD_VALUES, mode=PETSc.ScatterMode.REVERSE)
    fem.petsc.set_bc(b, [bc])

    # Solve linear problem
    solver.solve(b, p_h.vector)
    p_h.x.scatter_forward()

    # Update solution at previous time step (u_n)
    p_n.x.array[:] = p_h.x.array

    # Plot every 15th time step
    if i % 15 == 0:
        plot_function(t, p_h)

AppLayout(children=(VBox(children=(HTML(value='<h3>u(1639.344262295082)</h3>'), Dropdown(description='Colormap…

AppLayout(children=(VBox(children=(HTML(value='<h3>u(26229.508196721315)</h3>'), Dropdown(description='Colorma…

AppLayout(children=(VBox(children=(HTML(value='<h3>u(50819.67213114755)</h3>'), Dropdown(description='Colormap…

AppLayout(children=(VBox(children=(HTML(value='<h3>u(75409.83606557378)</h3>'), Dropdown(description='Colormap…

AppLayout(children=(VBox(children=(HTML(value='<h3>u(100000.00000000001)</h3>'), Dropdown(description='Colorma…